In [2]:
import numpy as np
import pandas as pd
import openai
import json
from openai.embeddings_utils import distances_from_embeddings
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"


from dotenv import load_dotenv
import os
load_dotenv()
# Load environment variable
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


import openai
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = OPENAI_API_KEY

df = pd.read_csv("embeddings.csv", index_col=0)
df["embeddings"] = df["embeddings"].apply(eval).apply(np.array)

while True:
    # Accept user input
    USER_QUESTION = input("Enter a movie description (or type 'exit' or 'q' to quit): ").strip()
    
    # Check if the user wants to exit
    if USER_QUESTION.lower() in ["exit", "q"]:
        print("Exiting the program. Goodbye!")
        break

    # Generate the embedding for the user question
    response = openai.Embedding.create(
        input=USER_QUESTION,
        engine=EMBEDDING_MODEL_NAME
    )

    # Extract the embeddings from the response
    question_embeddings = response["data"][0]["embedding"]

    # Calculate distances between the question embedding and dataset embeddings
    distances = distances_from_embeddings(
        question_embeddings,
        df["embeddings"],
        distance_metric="cosine"
    )

    # Add distances to the dataframe and sort by relevance
    df["distances"] = distances
    df.sort_values(by="distances", ascending=True, inplace=True)

    # Get the top 3 most relevant movies
    df_3movies = df.head(3)
    df_3movies_list = df_3movies[["Series_Title", "Overview"]].to_dict(orient="records")
    df_3movies_list_str = json.dumps(df_3movies_list)

    # Create the prompt
    prompt_template = f"""You are a movie recommendation system. You will be given a list of movies and their descriptions. Based on the descriptions, you will recommend the best movie that matches the user's question.
    The user question is: {USER_QUESTION}  
    The list of movies and their descriptions is: {df_3movies_list_str}
    you answer the question by giving the name of the movie and a short description of it.
    Answer:"""

    # Generate the response from the chatbot
    response = openai.Completion.create(
        model=COMPLETION_MODEL_NAME,
        prompt=prompt_template,
        max_tokens=1000
    )
    answer = response["choices"][0]["text"].strip()

    # Print the chatbot's answer
    print(f"Chatbot Answer: {answer}")
    print("-" * 50)


Chatbot Answer: The movie is Titanic. It is a classic love story set on the ill-fated Titanic ship. The main characters, an aristocrat and a poor artist, fall in love amidst the tragic events of the ship sinking. The movie is a must-see for those who enjoy romance and drama.
--------------------------------------------------
Exiting the program. Goodbye!
